In [2]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-06-28 18:43:35--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8000::154, 2606:50c0:8001::154, 2606:50c0:8002::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8000::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.1’

input.txt.1         100%[===================>]   1.06M  --.-KB/s    in 0.08s   

2025-06-28 18:43:36 (12.8 MB/s) - ‘input.txt.1’ saved [1115394/1115394]



In [3]:
with open('input.txt', 'r', encoding='utf8') as f:
    text = f.read()

In [4]:
print(len(text))

1115394


In [5]:
chars = sorted(list(set(text)))
vocab_size = len(chars)

In [6]:
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}

In [7]:
def encode(str):
    return [stoi[s] for s in str]
def decode(vec):
    return ''.join([itos[i] for i in vec])

In [8]:
sample = "Yo, what's up?"
print(encode(sample))
print(decode(encode(sample)))

[37, 53, 6, 1, 61, 46, 39, 58, 5, 57, 1, 59, 54, 12]
Yo, what's up?


In [9]:
import torch
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")

tensor([1.], device='mps:0')


In [10]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)

torch.Size([1115394]) torch.int64


### Train - Test Split

In [11]:
n = int(0.8*len(data))
train = data[:n]
val = data[n:]

In [24]:
batch_size = 8
block_size = 16
torch.manual_seed(42)

def get_batch(split):
    sample = train if split == 'train' else val
    index = torch.randint(len(sample)-block_size, (batch_size,))
    x = torch.stack([sample[i:i+block_size] for i in index])
    y = torch.stack([sample[i+1:i+block_size+1] for i in index])
    x, y = x.to(mps_device), y.to(mps_device)
    return x,y
    
xb, yb = get_batch('train')
print("xb:", xb)
print("yb:", yb)

xb: tensor([[61, 47, 50, 50, 12,  0,  0, 28, 27, 24, 21, 36, 17, 26, 17, 31],
        [43,  1, 51, 47, 45, 46, 58,  1, 39, 41, 58,  1, 58, 46, 43,  1],
        [56,  1, 53, 44,  1, 58, 56, 47, 40, 59, 52, 43, 57,  1, 58, 46],
        [ 1, 39,  1, 58, 39, 50, 50,  1, 44, 43, 50, 50, 53, 61,  1, 53],
        [53, 44,  1, 49, 52, 47, 45, 46, 58, 50, 63,  1, 58, 56, 47, 39],
        [56,  0, 32, 53,  1, 58, 39, 49, 43,  1, 53, 44, 44,  1, 57, 53],
        [59, 39, 56, 56, 43, 50, 50, 43, 42,  1, 61, 47, 58, 46,  1, 39],
        [56,  1, 58, 46, 43, 56, 43,  1, 47, 57,  1, 52, 53, 58, 46, 47]],
       device='mps:0')
yb: tensor([[47, 50, 50, 12,  0,  0, 28, 27, 24, 21, 36, 17, 26, 17, 31, 10],
        [ 1, 51, 47, 45, 46, 58,  1, 39, 41, 58,  1, 58, 46, 43,  1, 61],
        [ 1, 53, 44,  1, 58, 56, 47, 40, 59, 52, 43, 57,  1, 58, 46, 39],
        [39,  1, 58, 39, 50, 50,  1, 44, 43, 50, 50, 53, 61,  1, 53, 44],
        [44,  1, 49, 52, 47, 45, 46, 58, 50, 63,  1, 58, 56, 47, 39, 50],
      

In [13]:
for b in range(batch_size):
    for t in range(block_size):
        context = xb[b,:t+1]
        target = yb[b,t]
        print("Context: ", context, " target: ", target)

Context:  tensor([61])  target:  tensor(47)
Context:  tensor([61, 47])  target:  tensor(50)
Context:  tensor([61, 47, 50])  target:  tensor(50)
Context:  tensor([61, 47, 50, 50])  target:  tensor(12)
Context:  tensor([61, 47, 50, 50, 12])  target:  tensor(0)
Context:  tensor([61, 47, 50, 50, 12,  0])  target:  tensor(0)
Context:  tensor([61, 47, 50, 50, 12,  0,  0])  target:  tensor(28)
Context:  tensor([61, 47, 50, 50, 12,  0,  0, 28])  target:  tensor(27)
Context:  tensor([61, 47, 50, 50, 12,  0,  0, 28, 27])  target:  tensor(24)
Context:  tensor([61, 47, 50, 50, 12,  0,  0, 28, 27, 24])  target:  tensor(21)
Context:  tensor([61, 47, 50, 50, 12,  0,  0, 28, 27, 24, 21])  target:  tensor(36)
Context:  tensor([61, 47, 50, 50, 12,  0,  0, 28, 27, 24, 21, 36])  target:  tensor(17)
Context:  tensor([61, 47, 50, 50, 12,  0,  0, 28, 27, 24, 21, 36, 17])  target:  tensor(26)
Context:  tensor([61, 47, 50, 50, 12,  0,  0, 28, 27, 24, 21, 36, 17, 26])  target:  tensor(17)
Context:  tensor([61, 

#### How does the NN adapt to variable context length? 

### Bi-gram Language Model

In [ ]:
@torch.no_Grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'eval']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [14]:
import torch
import torch.nn as nn
from torch.nn import functional as F

class BigramLM(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx)
        if targets is None:
            loss = None
        else:
            
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
        
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:,-1,:] #only the last time step, becomes B, C
            probs = F.softmax(logits, dim =-1)
            idx_next = torch.multinomial(probs, num_samples=1) # B,1
            idx = torch.cat((idx, idx_next), dim=1) #B, T+1
        return idx

#cros_entropy = sigma(pi*ln(pi))
model = BigramLM(vocab_size)
m = model.to(mps_device)
logits, loss = model(xb, yb)
print(logits.shape)
print(loss) # how is this ln(1/65)

torch.Size([128, 65])
tensor(4.6852, grad_fn=<NllLossBackward0>)


In [15]:
print(decode(model.generate(idx=torch.zeros((1,1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


et'AcCaEDq
:sRW:SSyjXUMCg$mXxt.MK;gsdp:RQEbThnAfDP'FsIHhRW3&AfbaIydBXskCE:JzmJujoShDZLmGbnU;lyL.?XoS


In [17]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [22]:
batch_size = 32
for steps in range(15000):
    xb, yb = get_batch('train')
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

2.500779628753662


In [23]:
print(decode(model.generate(idx=torch.zeros((1,1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


CREE s y se
Vise che avothels t:
YOLLIftho m:
Y on,
ING t:
I sphiay ly
IORBryock
I'llerif ar uche sw
